# VacationPy
----

#### A

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv_file = "City Weather Data.csv"
city_weather_df = pd.read_csv(city_csv_file)
city_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Upernavik,72.7868,-56.1549,34.52,97,100,28.03,GL,1666472245
1,Harnai,30.1005,67.9374,68.22,23,0,4.05,PK,1666472245
2,Pec,42.6591,20.2883,59.13,51,15,6.24,XK,1666472245
3,Bluff,-46.6000,168.3333,56.80,69,26,10.47,NZ,1666472246
4,Dunedin,-45.8742,170.5036,55.80,74,6,1.01,NZ,1666471980


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Use the Lat and Lng as locations and Humidity as the weight.
# Remove rows missing data
city_weather_df = city_weather_df.dropna()
# convert humidity .astype float and store
humidity = city_weather_df["Humidity"].astype(float)
# max intensity in the heatmap based on highest humidity found in dataset
humidity_max = humidity.max()
# Store latitude and longitude in locations
locations = city_weather_df[['Lat', 'Lng']].astype(float)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity_max,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# new DataFrame to store narrowed down cities to fit weather conditions
narrow_city_df = city_weather_df

# set weather conditions
# max temperature < 85 and > 70
narrow_city_df = narrow_city_df.loc[(narrow_city_df['Max Temp'] < 85) & (narrow_city_df['Max Temp'] > 70)]
                                    
# wind speed less than 15 mph
narrow_city_df = narrow_city_df.loc[narrow_city_df['Wind Speed'] < 15]                            
                                    
# no clouds
narrow_city_df = narrow_city_df.loc[narrow_city_df['Cloudiness'] == 0]
                                     
# Remove rows missing data
narrow_city_df = narrow_city_df.dropna()

# reset index
narrow_city_df.reset_index(inplace=True)
del narrow_city_df['index']
                                     
narrow_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Harpālpur,25.2884,79.3347,72.18,57,0,1.95,IN,1666472250
1,Dwārka,22.2394,68.9678,81.93,50,0,10.16,IN,1666472254
2,Sirte,31.2089,16.5887,71.13,56,0,8.97,LY,1666472255
3,Bindki,26.0333,80.6000,70.41,66,0,3.60,IN,1666472257
4,Half Moon Bay,37.4636,-122.4286,72.41,61,0,14.97,US,1666472260


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/json
#   ?keyword=cruise
#   &location=-33.8670522%2C151.1957362
#   &radius=1500
#   &type=

In [6]:
hotel_df = narrow_city_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": 'hotel',
    "key": gkey
}

for index, row in hotel_df.iterrows():
    # get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row["Lng"]
    
    # add keyword to params dict
    params['location'] = f'{latitude}, {longitude}'
    
    # assemble url and make API request
    print(f'Retrieving Results for Index {index}: {row["City"]}.')
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    


Retrieving Results for Index 0: Harpālpur.
Closest hotel is Harpalpur.
------------
Retrieving Results for Index 1: Dwārka.
Closest hotel is Dwarka.
------------
Retrieving Results for Index 2: Sirte.
Closest hotel is Sirte.
------------
Retrieving Results for Index 3: Bindki.
Closest hotel is Bindki.
------------
Retrieving Results for Index 4: Half Moon Bay.
Closest hotel is Half Moon Bay.
------------
Retrieving Results for Index 5: Mopti.
Closest hotel is Tiaboly aldjouma.
------------
Retrieving Results for Index 6: Chimoio.
Closest hotel is Chimoio.
------------
Retrieving Results for Index 7: Sur.
Closest hotel is Sur.
------------
Retrieving Results for Index 8: Venice.
Closest hotel is Venice.
------------
Retrieving Results for Index 9: Nagrām.
Closest hotel is Samesee.
------------
Retrieving Results for Index 10: Verāval.
Closest hotel is Veraval.
------------
Retrieving Results for Index 11: Gainesville.
Closest hotel is Gainesville.
------------
Retrieving Results for Ind

In [7]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Harpālpur,25.2884,79.3347,72.18,57,0,1.95,IN,1666472250,Harpalpur
1,Dwārka,22.2394,68.9678,81.93,50,0,10.16,IN,1666472254,Dwarka
2,Sirte,31.2089,16.5887,71.13,56,0,8.97,LY,1666472255,Sirte
3,Bindki,26.0333,80.6000,70.41,66,0,3.60,IN,1666472257,Bindki
4,Half Moon Bay,37.4636,-122.4286,72.41,61,0,14.97,US,1666472260,Half Moon Bay


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))